### Ref:
- https://keras-team.github.io/keras-tuner/examples/helloworld/
- https://www.curiousily.com/posts/hackers-guide-to-hyperparameter-tuning/
- https://www.tensorflow.org/tutorials/keras/keras_tuner

In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=de662178272d5396e591e9eacfec05a8b27b5ab5a01e11823843d7ee36399709
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=f2892a79090892489d926d58f49b72318dc0ba292810332df48b5fa2e245fa30
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from kerastuner.tuners import RandomSearch

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
"""Basic case:
- We define a `build_model` function
- It returns a compiled model
- It uses hyperparameters defined on the fly
"""

def build_model(hp):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))

  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])),
                metrics=['accuracy'])
  return model

In [5]:
# Define tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='test_dir')

In [6]:
# Show stats
tuner.search_space_summary()

In [7]:
# Search for best params
tuner.search(x=x_train,
             y=y_train,
             epochs=5,
             validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3652 - accuracy: 0.8889 - val_loss: 0.0971 - val_accuracy: 0.9689
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2500 - accuracy: 0.9242 - val_loss: 0.0813 - val_accuracy: 0.9748
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2356 - accuracy: 0.9302 - val_loss: 0.0884 - val_accuracy: 0.9728
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2321 - accuracy: 0.9341 - val_loss: 0.0723 - val_accuracy: 0.9790
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2318 - accuracy: 0.9335 - val_loss: 0.0802 - val_accuracy: 0.9753
Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2683 - accuracy: 0.9204 - val_loss: 0.0888 - val_accuracy: 0.9732
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1871 - accuracy: 0.9471 - val_loss: 0.0714 - val_accuracy: 0.9793

Epoch 1/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1994 - accuracy: 0.9395 - val_loss: 0.0511 - val_accuracy: 0.9831
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0844 - accuracy: 0.9751 - val_loss: 0.0325 - val_accuracy: 0.9884
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0644 - accuracy: 0.9815 - val_loss: 0.0296 - val_accuracy: 0.9912
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0511 - accuracy: 0.9841 - val_loss: 0.0311 - val_accuracy: 0.9908
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0429 - accuracy: 0.9864 - val_loss: 0.0321 - val_accuracy: 0.9905
Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1904 - accuracy: 0.9418 - val_loss: 0.0514 - val_accuracy: 0.9838
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0840 - accuracy: 0.9755 - val_loss: 0.0421 - val_accuracy: 0.9861

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3894 - accuracy: 0.8852 - val_loss: 0.1000 - val_accuracy: 0.9688
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1337 - accuracy: 0.9608 - val_loss: 0.0609 - val_accuracy: 0.9802
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0966 - accuracy: 0.9717 - val_loss: 0.0465 - val_accuracy: 0.9851
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0797 - accuracy: 0.9762 - val_loss: 0.0401 - val_accuracy: 0.9864
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0697 - accuracy: 0.9793 - val_loss: 0.0352 - val_accuracy: 0.9877
Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3713 - accuracy: 0.8911 - val_loss: 0.0974 - val_accuracy: 0.9714
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1325 - accuracy: 0.9609 - val_loss: 0.0585 - val_accuracy: 0.9820

INFO:tensorflow:Oracle triggered exit


In [8]:
tuner.results_summary()

In [9]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_hps.get('learning_rate')

0.001

In [10]:
# Get best model
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [11]:
# Evaluate 
score = best_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.029560867697000504
Test accuracy: 0.9911999702453613


In [12]:
#Build the model with the optimal hyperparameters and train it on the data
model = None
model = tuner.hypermodel.build(best_hps)

model.fit(x=x_train,
             y=y_train,
             epochs=5,
             validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2130 - accuracy: 0.9356 - val_loss: 0.0505 - val_accuracy: 0.9827
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0899 - accuracy: 0.9728 - val_loss: 0.0377 - val_accuracy: 0.9879
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0646 - accuracy: 0.9800 - val_loss: 0.0344 - val_accuracy: 0.9897
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0529 - accuracy: 0.9837 - val_loss: 0.0291 - val_accuracy: 0.9903
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0457 - accuracy: 0.9855 - val_loss: 0.0279 - val_accuracy: 0.9920
